This notebook converts the data dictionaries from .csv to .json format and uploads it to the Mongodb collection `data_dictionary`.

In [52]:
# connect to mongo database
import pymongo
from db_config import *
db = pymongo.MongoClient(host=DB_HOST, port=DB_PORT)[DB_DATABASE]
db.authenticate(DB_USER, DB_PASSWORD)

True

### Load Files

In [ ]:
import pandas as pd
import numpy as np
import json
issue_csv = pd.read_csv("issue_dict.csv", dtype=str)
issdc_csv = pd.read_csv("issdc_dict.csv", dtype=str)
cvdisp_csv = pd.read_csv("cvdisp_dict.csv", dtype=str)

### Code to Convert to Json
This code converts the .csv files to .json

In [ ]:
# Convert csv files for the bva codes into json files
# Recursively scans columns left to right, depth-first
def make_dict(df):
    if df.shape[0] == 0 or df.shape[1] == 0:
        return

    code_name = df.columns[0]
    label_name = df.columns[1]
    current_code = code_name.split("_")[0]
    current_dict = {}
    child_dict = {}

    for code in df[code_name].unique():

        # Subset based on each code value
        df_subset = df[df[code_name] == code]
        label = df_subset[label_name].unique()
        if len(label) == 0:
            continue
        df_subset = df_subset.drop([code_name, label_name], axis=1)
        child_dict[code] = {"label": label[0]}

        # If columns remaining, append children
        if df_subset.shape[1] > 0 and df_subset.shape[0] > 0:
            grandchild = make_dict(df_subset)
            if len(grandchild) > 0:
                child_dict[code].update(**grandchild)

    # Finally append
    if len(child_dict) > 0:
        current_dict[current_code] = child_dict

    return current_dict

In [ ]:
# Compute
dictionaries = [{"name": "issue", "dict": make_dict(issue_csv)},
                {"name": "issdc", "dict": make_dict(issdc_csv)},
                {"name": "cvdisp", "dict": make_dict(cvdisp_csv)}]

In [65]:
# Example
print(json.dumps(dictionaries[1], indent=2))

{
  "name": "issdc",
  "dict": {
    "issdc": {
      "1": {
        "label": "Allowed (Board Code)"
      },
      "3": {
        "label": "Remanded (Board Code)"
      },
      "4": {
        "label": "Denied (Board Code)"
      },
      "5": {
        "label": "Vacated (Board Code)"
      },
      "6": {
        "label": "Dismissed/Withdrawn (Board Code)"
      },
      "8": {
        "label": "Dismissed Death (Board Code)"
      },
      "9": {
        "label": "Withdrawn (Obsolete Board Code, no longer used)"
      },
      "A": {
        "label": "Adv Allowed in Field (VBA Code)"
      },
      "B": {
        "label": "Benefits granted on Remand (VBA Code)"
      },
      "D": {
        "label": "Designation of Record (DOR) (Board Code)"
      },
      "E": {
        "label": "Death, Field (VBA Code)"
      },
      "F": {
        "label": "Withdrawn, Field (VBA Code)"
      },
      "G": {
        "label": "Closed for Failure to Submit Form 9 (VBA Code)"
      },
      "L": {
  

In [58]:
# Upload to mongodb
db.code_dictionary.insert_many(dictionaries)

In [83]:
# Look up dictionary
issue_dict = db.code_dictionary.find_one({"name": "issue"})
issue_dict["dict"]["issprog"]["2"]["isscode"]["4"]["label"] # Label of issprog=2, isscode=4

'Civil Service preference'